<a href="https://colab.research.google.com/github/jwlai-cloud/DataScience_Playground/blob/master/IP_range_to_CIDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [146]:
import pandas as pd
import numpy as np
import ipaddress

In [147]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
# df = pd.read_csv('/content/drive/MyDrive/bq-results-20210810-114917-et3sd45tr4vf/M247_UK_IP_merge_32x255x255.csv')
# df = pd.read_csv('/content/drive/MyDrive/bq-results-20210810-110223-ibeno2f7sf9/M247_UK_IP_merge_16x255x255.csv')
# df = pd.read_csv('/content/drive/MyDrive/bq-results-20210810-130309-mavm1uueootm/M247_UK_IP_merge_1x255x255.csv')
# df = pd.read_csv('/content/drive/MyDrive/bq-results-20210810-140638-ojld7whqhnoh/M247_UK_IP_merge_32x255x255_mask11.csv')
# df = pd.read_csv('/content/drive/MyDrive/bq-results-20210810-155907-g3t5kr5biy9v/M247_UK_IP_merge_1x255x255_mask24.csv')
# df = pd.read_csv('/content/drive/MyDrive/bq-results-20210810-163225-w3by6djzlgew/M247_UK_IP_merge_16x255x255_mask16.csv')
df = pd.read_csv('/content/drive/MyDrive/bq-results-20210812-152953-rbzq2ix3ow85/blackip_list.csv')


In [149]:
df = df[['SmartIP_first', 
         'SmartIP_last', 
         'ExtIP_first',
         'ExtIP_last'
]]

In [150]:
df.head()

,SmartIP_first,SmartIP_last,ExtIP_first,ExtIP_last
0,2.56.148.0,2.58.47.255,2.56.0.0,2.58.255.255
1,5.61.60.0,5.61.63.255,5.61.0.0,5.61.255.255
2,5.157.132.0,5.157.255.255,5.157.0.0,5.157.255.255
3,5.180.76.0,5.183.179.255,5.180.0.0,5.183.255.255
4,5.252.196.0,5.253.207.255,5.252.0.0,5.253.255.255


In [151]:
len(df)

172

In [152]:
raw_black_df = pd.read_csv('/content/drive/MyDrive/bq-results-20210812-162441-dkar6soeu92f/Raw_Blacklist_IP_Range.csv')

In [153]:
display(raw_black_df)

,ip_range,autonomous_system_number,autonomous_system_organization,isp,organization,country_name,cidr,IPint_from_str
0,31.204.48.0/23,9009,M247 Ltd,M247 Ltd,M247 Ltd,NaN,23,533475328
1,37.120.144.0/20,9009,M247 Ltd,M247 Ltd,M247 Ltd,NaN,20,628658176
2,45.83.88.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,NaN,22,760436736
3,45.84.120.0/22,9009,M247 Ltd,M247 Ltd,M247 Ltd,NaN,22,760510464
4,45.153.236.0/23,9009,M247 Ltd,M247 Ltd,M247 Ltd,NaN,23,765062144
...,...,...,...,...,...,...,...,...
788,38.206.128.0/22,9009,M247 Ltd,Cogent Communications,M247 Ltd,NaN,22,651067392
789,154.30.240.0/21,9009,M247 Ltd,Cogent Communications,M247 Ltd,NaN,21,2585718784
790,212.102.156.0/22,9009,M247 Ltd,Men Danil Valentinovich,M247 Ltd,NaN,22,3563494400
791,161.123.8.0/22,9009,M247 Ltd,BEAMSPEED,Beamspeed LLC,United Kingdom,22,2709194752


In [154]:
raw_black_network = pd.Series([ipaddress.IPv4Network(item) for item in raw_black_df['ip_range']])

In [155]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/bq-results-20210810-131723-g28anqrqoh0v/whitelist_ip.csv')

In [156]:
len(whitelist_df)

131484

In [157]:
white_ipint = whitelist_df['ip_range'].values

In [158]:
IP_Metric_TG_df = pd.read_csv('/content/drive/MyDrive/bq-results-20210812-094526-4ujpsfhuxwwg/IP_Metrics_TG.csv')

In [159]:
len(IP_Metric_TG_df)

155776

In [160]:
ip_range=[]

In [161]:
# for row in df.itertuples():
#   ip_range.append([str(ipaddr)  for ipaddr in 
#                           ipaddress.summarize_address_range(ipaddress.IPv4Address(row.SmartIP_first), 
#                                                             ipaddress.IPv4Address(row.SmartIP_last)) 
#                           ])
  
for row in df.itertuples():
  ip_range.append([(ipaddr, row.ExtIP_first,row.ExtIP_last)   for ipaddr in 
                          ipaddress.summarize_address_range(ipaddress.IPv4Address(row.ExtIP_first), 
                                                            ipaddress.IPv4Address(row.ExtIP_last)) 
                          ])

In [162]:
ip_range[0]

[(IPv4Network('2.56.0.0/15'), '2.56.0.0', '2.58.255.255'),
 (IPv4Network('2.58.0.0/16'), '2.56.0.0', '2.58.255.255')]

In [163]:
ip_range_flat = [(ipaddr,compr_first,compr_last ) for item in ip_range for (ipaddr, compr_first, compr_last ) in item ]

In [164]:
print(f"The total number of IP CIDR ranges after compression is {len(ip_range_flat)}")

The total number of IP CIDR ranges after compression is 300


In [165]:
ip_range_flat_str = [str(ipr[0]) for ipr in ip_range_flat]
ip_range_flat_min_ipstr = [str(ipr[0][0]) for ipr in ip_range_flat]
ip_range_flat_max_ipstr = [str(ipr[0][-1]) for ipr in ip_range_flat]
ip_range_flat_min_ipint = [int(ipr[0][0]) for ipr in ip_range_flat]
ip_range_flat_max_ipint = [int(ipr[0][-1]) for ipr in ip_range_flat]
ip_range_flat_min_compripstr = [ipr[1] for ipr in ip_range_flat]
ip_range_flat_max_compripstr = [ipr[2] for ipr in ip_range_flat]
ip_range_flat_int_pair = [(int(ipr[0][0]), int(ipr[0][-1])) for ipr in ip_range_flat]

In [166]:
cidr_ip_range_df = pd.DataFrame({
    "IP_range_str": ip_range_flat_str,
    "ExtIP_first": ip_range_flat_min_ipstr,
    "ExtIP_last": ip_range_flat_max_ipstr,
    "ExtIP_int_first": ip_range_flat_min_ipint,
    "ExtIP_int_last": ip_range_flat_max_ipint,
    "ComprIP_first": ip_range_flat_min_compripstr,
    "ComprIP_last": ip_range_flat_max_compripstr,
    "IP_Pair": ip_range_flat_int_pair
})

In [167]:
cidr_ip_range_df.head()

,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,IP_Pair
0,2.56.0.0/15,2.56.0.0,2.57.255.255,37224448,37355519,2.56.0.0,2.58.255.255,"(37224448, 37355519)"
1,2.58.0.0/16,2.58.0.0,2.58.255.255,37355520,37421055,2.56.0.0,2.58.255.255,"(37355520, 37421055)"
2,5.61.0.0/16,5.61.0.0,5.61.255.255,87883776,87949311,5.61.0.0,5.61.255.255,"(87883776, 87949311)"
3,5.157.0.0/16,5.157.0.0,5.157.255.255,94175232,94240767,5.157.0.0,5.157.255.255,"(94175232, 94240767)"
4,5.180.0.0/14,5.180.0.0,5.183.255.255,95682560,95944703,5.180.0.0,5.183.255.255,"(95682560, 95944703)"


In [168]:
num_whiteIPrange = len(whitelist_df)

In [169]:
def is_white_inside(col):
  hits_sum = whitelist_df[(col[0]<=whitelist_df['ip_range']) & (whitelist_df['ip_range']<=col[1])].ip_range.count()
  hits_rate = hits_sum/num_whiteIPrange
  return (hits_sum, f"{hits_rate:.3%}")

In [170]:
def metric_inside(col):
  metrics = tuple(IP_Metric_TG_df[(col[0]<=IP_Metric_TG_df['ip_range_int']) & (IP_Metric_TG_df['ip_range_int']<=col[1])]
                  [['ip_range',
                    'total_click', 
                    'success_click',
                    'invalid_click',
                    'count_ppc_invalid_click_known_reason',
                    'watchlist_count',
                    'total_conversion_event_journey']]
                  .agg({'ip_range': 'count', 
                        'total_click': 'sum', 
                        'success_click': 'sum',
                        'invalid_click': 'sum',
                        'count_ppc_invalid_click_known_reason': 'sum',
                        'watchlist_count': 'sum',
                        'total_conversion_event_journey': 'sum'}))
  return metrics

In [171]:
# raw_black_network[0].overlaps() ipaddress.IPv4Network.overlaps() ('31.204.43.0/24'))
def is_raw_black_inside(col):
  hits_sum = raw_black_network.apply(lambda x: x.overlaps(col)).sum()
  return hits_sum

In [172]:
cidr_ip_range_df['network']= cidr_ip_range_df['IP_range_str'].apply(ipaddress.IPv4Network)

In [173]:
cidr_ip_range_df['Raw_BlackIP_hits'] = cidr_ip_range_df['network'].apply(is_raw_black_inside)

In [174]:
cidr_ip_range_df['WhiteIP_hits'] = cidr_ip_range_df.IP_Pair.apply(is_white_inside)

In [175]:
cidr_ip_range_df[['WhiteIP_hits_total', 'WhiteIP_hits_rate']] = pd.DataFrame([item for item in cidr_ip_range_df['WhiteIP_hits']])

In [176]:
cidr_ip_range_df['IP_Metric_QC'] = cidr_ip_range_df.IP_Pair.apply(metric_inside)

In [177]:
cidr_ip_range_df[['Metric_hits_total', 
                  'Total_click', 
                  'Success_click',
                  'Invalid_click',
                  'Count_ppc_invalid_click_known_reason',
                  'Watchlist_count',
                  'Total_conversion_event_journey']] = pd.DataFrame([item for item in cidr_ip_range_df['IP_Metric_QC']])

In [178]:
cidr_ip_range_df.drop(columns=['network', 'IP_Pair','WhiteIP_hits','IP_Metric_QC'], inplace=True)

In [179]:
cidr_ip_range_df.sort_values(by='WhiteIP_hits_total', ascending=False).head()

,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,Raw_BlackIP_hits,WhiteIP_hits_total,WhiteIP_hits_rate,Metric_hits_total,Total_click,Success_click,Invalid_click,Count_ppc_invalid_click_known_reason,Watchlist_count,Total_conversion_event_journey
95,89.240.0.0/13,89.240.0.0,89.247.255.255,1508900864,1509425151,89.238.0.0,89.249.255.255,0,973,0.740%,921.0,28691.0,25635.0,3056.0,2.0,339.0,5162.0
252,194.64.0.0/10,194.64.0.0,194.127.255.255,3258974208,3263168511,194.31.0.0,194.127.255.255,14,695,0.529%,771.0,12411.0,6491.0,5920.0,5352.0,4656.0,403.0
216,185.128.0.0/9,185.128.0.0,185.255.255.255,3112173568,3120562175,185.75.0.0,185.255.255.255,93,193,0.147%,606.0,22787.0,13330.0,9457.0,6523.0,3961.0,2104.0
296,217.138.0.0/15,217.138.0.0,217.139.255.255,3649699840,3649830911,217.138.0.0,217.151.255.255,6,154,0.117%,161.0,1242.0,735.0,507.0,456.0,43.0,581.0
53,77.84.0.0/14,77.84.0.0,77.87.255.255,1297350656,1297612799,77.81.0.0,77.90.255.255,0,115,0.087%,112.0,4026.0,3712.0,314.0,0.0,53.0,517.0


In [182]:
cidr_ip_range_df.query('Raw_BlackIP_hits > 0', inplace=True)

In [183]:
%load_ext google.colab.data_table
cidr_ip_range_df

,IP_range_str,ExtIP_first,ExtIP_last,ExtIP_int_first,ExtIP_int_last,ComprIP_first,ComprIP_last,Raw_BlackIP_hits,WhiteIP_hits_total,WhiteIP_hits_rate,Metric_hits_total,Total_click,Success_click,Invalid_click,Count_ppc_invalid_click_known_reason,Watchlist_count,Total_conversion_event_journey
0,2.56.0.0/15,2.56.0.0,2.57.255.255,37224448,37355519,2.56.0.0,2.58.255.255,1,0,0.000%,7.0,146.0,122.0,24.0,3.0,2.0,0.0
1,2.58.0.0/16,2.58.0.0,2.58.255.255,37355520,37421055,2.56.0.0,2.58.255.255,1,0,0.000%,6.0,167.0,12.0,155.0,152.0,12.0,0.0
2,5.61.0.0/16,5.61.0.0,5.61.255.255,87883776,87949311,5.61.0.0,5.61.255.255,1,8,0.006%,10.0,305.0,290.0,15.0,0.0,0.0,0.0
3,5.157.0.0/16,5.157.0.0,5.157.255.255,94175232,94240767,5.157.0.0,5.157.255.255,7,0,0.000%,7.0,61.0,53.0,8.0,0.0,0.0,3.0
4,5.180.0.0/14,5.180.0.0,5.183.255.255,95682560,95944703,5.180.0.0,5.183.255.255,7,6,0.005%,20.0,425.0,195.0,230.0,210.0,76.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,217.9.0.0/16,217.9.0.0,217.9.255.255,3641245696,3641311231,217.9.0.0,217.9.255.255,1,0,0.000%,16.0,4726.0,75.0,4651.0,4648.0,4336.0,11.0
295,217.64.0.0/16,217.64.0.0,217.64.255.255,3644850176,3644915711,217.64.0.0,217.64.255.255,1,0,0.000%,5.0,17.0,11.0,6.0,5.0,0.0,0.0
296,217.138.0.0/15,217.138.0.0,217.139.255.255,3649699840,3649830911,217.138.0.0,217.151.255.255,6,154,0.117%,161.0,1242.0,735.0,507.0,456.0,43.0,581.0
298,217.144.0.0/13,217.144.0.0,217.151.255.255,3650093056,3650617343,217.138.0.0,217.151.255.255,2,29,0.022%,62.0,1852.0,407.0,1445.0,1383.0,322.0,30.0


In [184]:
cidr_ip_range_df.describe()

,ExtIP_int_first,ExtIP_int_last,Raw_BlackIP_hits,WhiteIP_hits_total,Metric_hits_total,Total_click,Success_click,Invalid_click,Count_ppc_invalid_click_known_reason,Watchlist_count,Total_conversion_event_journey
count,2.660000e+02,2.660000e+02,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000
mean,2.144487e+09,2.144733e+09,2.981203,7.372180,14.695489,517.556391,213.763158,303.793233,277.631579,204.902256,32.590226
std,1.026562e+09,1.026632e+09,6.642035,45.995412,64.291445,1968.426047,1053.565855,1152.028075,1056.674520,911.384784,153.074557
min,3.722445e+07,3.735552e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.399620e+09,1.399685e+09,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.828028e+09,1.828094e+09,1.000000,0.000000,2.000000,9.500000,0.500000,1.000000,0.000000,0.000000,0.000000
75%,3.169042e+09,3.169157e+09,3.000000,1.000000,7.000000,162.000000,56.750000,36.250000,19.250000,4.000000,2.000000
max,3.653566e+09,3.653632e+09,93.000000,695.000000,771.000000,22787.000000,13330.000000,9457.000000,9057.000000,8390.000000,2104.000000


In [188]:
print(f"Final false positive rate is {cidr_ip_range_df.WhiteIP_hits_total.sum()/num_whiteIPrange:.3%}")

Final false positive rate is 1.491%


In [180]:
cidr_ip_range_df.to_csv('cidr_iprange_qc.csv')